In [ ]:
import pandas as pd
import plotly.express as px

def get_trendline_df(var_name,fig):
    trend_results = px.get_trendline_results(fig)
    formulas = []
    for idx in big_site_df[var_name].unique():
        res = trend_results.loc[trend_results[var_name] == idx, 'px_fit_results']
        if not res.empty:
            model = res.iloc[0]
            params = model.params
            intercept,slope = params
            formula = f'y = {slope:.4f}·x + {intercept:.4f}'
            formulas.append({
                var_name: idx,
                'Slope': slope,
                'Intercept': intercept,
                'Formula': formula,
                'R-squared': model.rsquared
            })

    return pd.DataFrame(formulas)

# Load the combined CSV
df = pd.read_csv('power_summary_combined_all.csv')
# df = df[df['Total kWh'] > 1000]  # Filter out rows with Total kWh > 1000
big_site_df = df[~df['Name'].str.contains('#')]  # Filter out rows with Total kWh > 1000
small_site_df = df[df['Name'].str.contains('#')] 


In [ ]:
# Create scatter plot with OLS trendlines per address (Name)
fig = px.scatter(
    big_site_df,
    x='Total kWh',
    y='Total',
    color='Name',
    trendline='ols',
    trendline_scope='trace',
    labels={
        # 'Total KWh': 'Total kWh',
        'Total': 'Total Cost',
        # 'Name': 'Name'
    },
    title='Total Cost vs Total kWh with per Site',
        height= 600,
    width= 800
)
fig.show()

site_formulas = get_trendline_df('Name',fig)


In [3]:
annual_average = big_site_df.groupby('Name').agg({'Total kWh': 'mean', 'Total': 'mean'}).reset_index()

mask = annual_average['Total kWh'] < 5000


annual_average.loc[mask,'Rate Code'] = 'D200'
annual_average.loc[~mask,'Rate Code'] = 'D300'
rate_code_mapping = annual_average.set_index('Name').to_dict('dict')['Rate Code']

In [4]:
d200_sites_n  = mask.sum()
d300_sites_n  = (~mask).sum()

f"Billing data for {d200_sites_n} sites with D200 rate code and {d300_sites_n} sites with D300 rate code was used to calculate trendlines using ordinary least squares."

'Billing data for 2 sites with D200 rate code and 3 sites with D300 rate code was used to calculate trendlines using ordinary least squares.'

In [ ]:
# Create scatter plot with OLS trendlines per address (Name)
big_site_df['Rate Code'] = big_site_df['Name'].map(rate_code_mapping)
fig = px.scatter(
    big_site_df,
    x='Total kWh',
    y='Total',
    color='Rate Code',
    trendline='ols',
    trendline_scope='trace',
    labels={
        # 'Total KWh': 'Total kWh',
        'Total': 'Total Cost',
        # 'Name': 'Name'
    },
    title='Total Cost vs Total KWh per Rate Code',
    height= 600,
    width= 800
)
fig.show()

rate_code_formulas = get_trendline_df('Rate Code',fig)

/tmp/ipykernel_7809/3510004705.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [6]:
rate_code_formulas

,Rate Code,Slope,Intercept,Formula,R-squared
0,D200,0.125676,57.596387,y = 0.1257·x + 57.5964,0.993307
1,D300,0.123478,403.275106,y = 0.1235·x + 403.2751,0.958736


In [7]:
model.rsquared

NameError: name 'model' is not defined